In [12]:
import json
import csv
import sys
import datetime
import os

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

from shapely.geometry import shape, Point
from shapely.geos import TopologicalError

from multiprocessing import Pool
pool = Pool() 

In [13]:
def build_zip3_polygons(jsonfile):
    # load GeoJSON file containing sectors
    with open(jsonfile, 'r') as zip3f:
        zip3js = json.load(zip3f)

    # Builds list of zip3 polygons. Corrects invalid shape geometry.
    zip3_polygon_list = []
    for zip3_feature in zip3js['features']:
        zip3_polygon = shape(zip3_feature['geometry'])
        if zip3_polygon.is_valid:
            zip3_polygon_list.append( (zip3_polygon, zip3_feature["properties"]['ZIP']) ) 
        else:
            zip3_polygon_list.append( (zip3_polygon.buffer(0), zip3_feature["properties"]['ZIP']) )

    return zip3_polygon_list

In [14]:
# Downloads csv files, places in csv folder
def grab_files(dl):
    file_list = []
    file_url = 'http://aqsdr1.epa.gov/aqsweb/aqstmp/airdata/'
    if not os.path.isdir('csv'):
        os.system('mkdir csv')
    for i in xrange(6):
        file_list.append(dl + str(i) + '.csv')
    for fl in file_list:
        if not os.path.isfile(os.path.join('csv',fl)):
            zipfile = fl[:-4] + '.zip'
            os.system('wget ' + file_url + zipfile)
            os.system('unzip ' + zipfile + ' -d csv')
            os.system('rm -rf ' + zipfile)
    return file_list

In [15]:
# Loads data into dictionary of coordinates
# Output: { ( lon, lat ) : { date : value } }
def load_data_file(filename):
    coord_dict = {}
    for file in filename:
        csv_file = csv.DictReader(open(os.path.join('csv',file)))
        for idx, row in enumerate(csv_file):
            #sys.stdout.write('\r')
            #sys.stdout.write("%d" % (idx))
            #sys.stdout.flush()
            lon = float(row["Longitude"])
            lat = float(row["Latitude"])
            coord = (lon,lat)
            date = datetime.datetime.strptime(row['Date Local'], '%Y-%m-%d').date()
            value = float(row['1st Max Value'])
            try:
                coord_dict[coord][date] = value
            except KeyError:
                coord_dict[coord] = {date:value}
            
    return coord_dict

In [16]:
def state_for_zip(zip3):
    state_hash = {
        'Mississippi': ['386-397'],
        'Oklahoma': ['730-731', '734-741', '743-749'],
        'Delaware': ['197-199'],
        'Minnesota': ['550-551', '553-566'],
        'Alaska': ['995-999'],
        'Illinois': ['600-620', '622-629'],
        'Arkansas': ['716-729'],
        'New Mexico': ['870-875', '877-884'],
        'Indiana': ['460-470', '472-479'],
        'Maryland': ['206-212', '214-219'],
        'Louisiana': ['700-701', '703-708', '710-714'],
        'Texas': ['733-733', '750-799', '885-885'],
        'Wyoming': ['820-831'],
        'Tennessee': ['370-385'],
        'Armed Forces Americas': ['340-340'],
        'Iowa': ['500-528'],
        'Arizona': ['850-850', '852-853', '855-857', '859-860', '863-865'],
        'Guam': ['969-969'],
        'Michigan': ['480-499'],
        'Kansas': ['660-662', '664-679'],
        'Utah': ['840-847'],
        'Virginia': ['201-201', '220-246'],
        'Oregon': ['970-979'],
        'Puerto Rico': ['006-007', '009-009'],
        'Connecticut': ['060-069'],
        'Virgin Islands': ['008-008'],
        'New Hampshire': ['030-038'],
        'Massachusetts': ['010-027', '055-055'],
        'West Virginia': ['247-268'],
        'South Carolina': ['290-299'],
        'California': ['900-908', '910-928', '930-961'],
        'Wisconsin': ['530-532', '534-535', '537-549'],
        'Vermont': ['050-054', '056-059'],
        'Georgia': ['300-319', '398-399'],
        'North Dakota': ['567-567', '580-588'],
        'Pennsylvania': ['150-196'],
        'Armed Forces Europe*': ['090-099'],
        'Florida': ['320-339', '341-342', '344-344', '346-347', '349-349'],
        'Hawaii': ['967-968'],
        'Kentucky': ['400-427', '471-471'],
        'Rhode Island': ['028-029'],
        'Nebraska': ['680-681', '683-693'],
        'Armed Forces Pacific': ['962-966'],
        'District of Columbia': ['200-200', '202-205'],
        'Ohio': ['430-459'],
        'Alabama': ['350-352', '354-369'],
        'South Dakota': ['570-577'],
        'Colorado': ['800-816'],
        'Idaho': ['832-838'],
        'New Jersey': ['070-089'],
        'Missouri': ['630-631', '633-641', '644-658'],
        'Washington': ['980-986', '988-994'],
        'North Carolina': ['270-289'],
        'New York': ['005-005', '100-149'],
        'Montana': ['590-599'],
        'Nevada': ['889-891', '893-895', '897-898'],
        'Maine': ['039-049']}
    
    for state in state_hash:
        for interval in state_hash[state]:
            lower = interval.split("-")[0]
            upper = interval.split("-")[1]
            if lower==upper and lower==zip3:
                return state
            
            if int(lower) <= int(zip3) and int(zip3) < int(upper):
                return state
    
    return None

In [17]:
# Takes input as longitude first, then latitude
# West is negative
# Returns zip3
def point_check(lon,lat, zip3_polygon_list):
    point = Point(lon, lat)
    # check each zip 3 polygon to see if it contains the point
    for poly in zip3_polygon_list:
        if poly[0].contains(point):
            #print 'Found in zip3:', zip3_feature['properties']['ZIP']
            #print
            return str(poly[1])
    return None

In [18]:
# Populates zip3 and state dict for value lookups
# zip3_dict  := { 'zip3'  : { ( lon, lat ) : { date : value } } }
# state_dict := { 'State' : { ( lon, lat ) : { date : value } } }
def populate_dicts(coord_dict,zip3_polygon_list):
    zip3_dict = {}
    state_dict = {}
    for coord in coord_dict.keys():
        try:
            zip3 = point_check(coord[0],coord[1],zip3_polygon_list)
        except TopologicalError:
            zip3 = None

        try:
            zip3_dict[str(zip3)][coord] = coord_dict[coord]
        except KeyError:
            zip3_dict[str(zip3)] = {coord:coord_dict[coord]}

        try:
            state = state_for_zip(zip3)
        except TypeError:
            state = 'None'

        try:
            state_dict[state][coord] = coord_dict[coord]
        except KeyError:
            state_dict[state] = {coord:coord_dict[coord]}

    return zip3_dict, state_dict

In [19]:
# Prints zip3 graph
def zip3_graph(zip3,begin,end,zip3_dict):
    timeline_dict = {}
    graph_x_list = []
    graph_y_list = []
    begin_date = datetime.datetime.strptime(begin,'%m-%d-%Y').date()
    end_date = datetime.datetime.strptime(end,'%m-%d-%Y').date()
    try:
        for coord in zip3_dict[zip3].keys():
            for date in zip3_dict[zip3][coord]:
                if date >= begin_date and date <= end_date:
                    value = zip3_dict[zip3][coord][date]
                    lon = coord[0] 
                    lat = coord[1]

                    try:
                        sub_total = timeline_dict[date][0]*timeline_dict[date][1]
                        num_pts = timeline_dict[date][1]+1
                        timeline_dict[date][0] = (sub_total + value)/num_pts
                        timeline_dict[date][1] = num_pts
                        timeline_dict[date][2].append( { ( lon, lat ) : value } )
                    except KeyError:
                        timeline_dict[date] = [ value, 1, [ { ( lon, lat ) : value } ] ]

        for date in sorted(timeline_dict.keys()):
            graph_x_list.append(date)
            graph_y_list.append(timeline_dict[date][0])

        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d/%Y'))
        plt.plot(graph_x_list,graph_y_list,marker='o')
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title('Values in zip3 '+str(zip3)+' per Date')
        plt.gcf().autofmt_xdate()
        plt.show()
        
    except KeyError:
        print "No data for zip3 " + str(zip3)

    # { Date : [Average, num pts, [{(lat,long):value}...]] }
    #return timeline_dict

In [20]:
# Prints state graph
def state_graph(state,begin,end,state_dict):
    timeline_dict = {}
    graph_x_list = []
    graph_y_list = []
    begin_date = datetime.datetime.strptime(begin,'%m-%d-%Y').date()
    end_date = datetime.datetime.strptime(end,'%m-%d-%Y').date()
    try:
        for coord in state_dict[state].keys():
            for date in state_dict[state][coord]:
                if date >= begin_date and date <= end_date:
                    value = state_dict[state][coord][date]
                    lon = coord[0] 
                    lat = coord[1]

                    try:
                        sub_total = timeline_dict[date][0]*timeline_dict[date][1]
                        num_pts = timeline_dict[date][1]+1
                        timeline_dict[date][0] = (sub_total + value)/num_pts
                        timeline_dict[date][1] = num_pts
                        timeline_dict[date][2].append( { ( lon, lat ) : value } )
                    except KeyError:
                        timeline_dict[date] = [ value, 1, [ { ( lon, lat ) : value } ] ]

        for date in sorted(timeline_dict.keys()):
            graph_x_list.append(date)
            graph_y_list.append(timeline_dict[date][0])

        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d/%Y'))
        plt.plot(graph_x_list,graph_y_list,marker='o')
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title('Values in '+str(state)+' per Date')
        plt.gcf().autofmt_xdate()
        plt.show()
    
    except KeyError:
        print "No data for state " + str(state)

    # { Date : [Average, num pts, [{(lat,long):value}...]] }
    #return timeline_dict

In [21]:
# Calculates average value of points in specific radius
# [ Average, num pts,  { nearest k ( lon, lat ) : min_dist_km } , distance [km], [{(lat,long):value}...]]
# Example:

#print range_check(-105.002332,39.746389,20,'01-01-2010','12-31-2015',ozone_state_dict)

#Output:
#[0.046, 10, {(-104.987625, 39.751184): 1.3496702992280396}, 0.015468932542359015, 
#[{(-105.13948, 39.638781): 0.058}, {(-105.099973, 39.800333): 0.031}, {(-104.94984, 39.838119): 0.06},
#{(-104.987625, 39.751184): 0.053}, {(-105.070358, 39.534488): 0.061}, {(-104.998113, 39.704005): 0.011},
#{(-105.177989, 39.743724): 0.061}, {(-104.957193, 39.567887): 0.025}, {(-105.00518, 39.77949): 0.049},
#{(-105.030681, 39.751761): 0.051}]]

# 0.046 is the average of all points
# 10 is the number of points
# {(-104.987625, 39.751184): 1.3496702992280396} is the nearest point to the provided lon, lat
# (-104.987625, 39.751184) is the coordinates of the nearest point
# 1.3496702992280396 is the value of the nearest point
# 0.015468932542359015 is the distance of the nearest point to the provided lon, lat
# [{(-105.13948, 39.638781): 0.058}, {(-105.099973, 39.800333): 0.031}, {(-104.94984, 39.838119): 0.06},
# {(-104.987625, 39.751184): 0.053}, {(-105.070358, 39.534488): 0.061}, {(-104.998113, 39.704005): 0.011},
# {(-105.177989, 39.743724): 0.061}, {(-104.957193, 39.567887): 0.025}, {(-105.00518, 39.77949): 0.049},
# {(-105.030681, 39.751761): 0.051}]] is the list of points used in calculations
# (-105.13948, 39.638781) is the coordinates of the point
# 0.058 is the value of the point
def range_check(lon,lat,radius,begin,end,state_dict):
    input_point = Point(lon,lat)
    
    # Convert radius (in km) to relative lon/lat coordinate distance
    # Technically, it doesn't calculate out this neatly due to the curvature of the earth
    # But, for small radius calculations, it should be okay
    ##Distance calculation: about 5 km (used site http://www.movable-type.co.uk/scripts/latlong.html)
    #x1 = 5
    #y1 = Point(-104.94402,39.743).distance(Point(-105.002332,39.746389))
    ##Distance calculation: about 1 km
    #x2 = 1
    #y2 = Point(-105.01317,39.743).distance(Point(-105.002332,39.746389))
    #m = (y2-y1)/(x2-x1)
    ## y = mx+b
    #b = y2-m*x2
    #print "y = " + str(m) + "x + " + str(b)
    #y = 0.0117637226171x + -0.000408214482298
    
    coordinate_distance = 0.0117637226171*radius + -0.000408214482298

    local_list = []
    point_list = []
    min_distance = np.inf
    begin_date = datetime.datetime.strptime(begin,'%m-%d-%Y').date()
    end_date = datetime.datetime.strptime(end,'%m-%d-%Y').date()
    for state in state_dict.keys():
        try:
            for coord in state_dict[state]:
                for date in state_dict[state][coord]:
                    if date >= begin_date and date <= end_date:
                        lon = coord[0]
                        lat = coord[1]
                        value = state_dict[state][coord][date]
                        entry_point = Point(lon,lat)
                        if input_point.distance(entry_point) <= coordinate_distance:
                            local_list.append(value)
                            point_list.append( { ( lon, lat ) : value } )
                            if input_point.distance(entry_point) < min_distance:
                                min_distance = input_point.distance(entry_point)
                                min_dist_km = (min_distance + 0.000408214482298)/0.0117637226171
                                min_point = { ( lon, lat ) : min_dist_km }
        except KeyError:
            print "No data for state " + str(state)
            return None
    try:
        average = sum(local_list)/len(local_list)
    except ZeroDivisionError:
        return None
    
    num_pts = len(local_list)
    
    return [ average, num_pts, min_point, min_distance, point_list ]

In [ ]:
if __name__ == "__main__":
    jsonfile = 'zip3.json'
    zip3_database = build_zip3_polygons(jsonfile)
    
    # file name prefixes to be downloaded from http://aqsdr1.epa.gov/aqsweb/aqstmp/airdata/download_files.html
    ozone = 'daily_44201_201'
    so2   = 'daily_42401_201'
    co    = 'daily_42101_201'
    no2   = 'daily_42602_201'
    
    ozone_file_list = grab_files(ozone)
    so2_file_list = grab_files(so2)
    co_file_list = grab_files(co)
    no2_file_list = grab_files(no2)
    
    # This part takes time to execute
    # I think I parallelized this section....
    # Original code execution:
    # coord_list = load_data_file( ozone_file_list )
    # ozone_zip3_dict, ozone_state_dict = populate_dicts( coord_list )
    multiprocessing_enable = True
    if multiprocessing_enable == True:
        result1 = pool.apply_async( load_data_file, [ ozone_file_list ] )
        result2 = pool.apply_async( load_data_file, [ so2_file_list   ] )
        result3 = pool.apply_async( load_data_file, [ co_file_list    ] )
        result4 = pool.apply_async( load_data_file, [ no2_file_list   ] )

        ozone_coord_list = result1.get(timeout=3600)
        so2_coord_list   = result2.get(timeout=3600)
        co_coord_list    = result3.get(timeout=3600)
        no2_coord_list   = result4.get(timeout=3600)

        result1 = pool.apply_async( populate_dicts, [ ozone_coord_list, zip3_database ] )
        result2 = pool.apply_async( populate_dicts, [ so2_coord_list, zip3_database   ] )
        result3 = pool.apply_async( populate_dicts, [ co_coord_list, zip3_database    ] )
        result4 = pool.apply_async( populate_dicts, [ no2_coord_list, zip3_database   ] )

        ozone_zip3_dict, ozone_state_dict = result1.get(timeout=50)
        so2_zip3_dict,   so2_state_dict   = result2.get(timeout=50)
        co_zip3_dict,    co_state_dict    = result3.get(timeout=50)
        no2_zip3_dict,   no2_state_dict   = result4.get(timeout=50)

    else:
        coord_list = load_data_file( ozone_file_list )
        ozone_zip3_dict, ozone_state_dict = populate_dicts( coord_list, zip3_database )
        coord_list = load_data_file( ozone_file_list )
        so2_zip3_dict, so2_state_dict = populate_dicts( coord_list, zip3_database )
        coord_list = load_data_file( ozone_file_list )
        co_zip3_dict, co_state_dict = populate_dicts( coord_list, zip3_database )
        coord_list = load_data_file( ozone_file_list )
        no2_zip3_dict, no2_state_dict = populate_dicts( coord_list, zip3_database )

Self-intersection at or near point -78.778527906282719 43.017714979730926
Self-intersection at or near point -81.12325396391924 37.681599524236759
Self-intersection at or near point -117.80283229921857 33.706457898118025


In [ ]:
print point_check(-105.002332,39.746389,zip3_database) # UCDenver
print point_check(-104.7,39.69,zip3_database)          # Aurora, CO
print point_check(-81.55,28.37,zip3_database)          # Epcot in Disney World
print point_check(-77.04,38.89,zip3_database)          # Washington Monument
print point_check(-1.826,51.18,zip3_database)          # Stonehenge, which is not found

In [ ]:
#print range_check(-105.002332,39.746389,1.4,'01-01-2010','12-31-2015',ozone_state_dict)
#print range_check(-105.002332,39.746389,20,'01-01-2010','12-31-2015',ozone_state_dict)
        
state_graph('Colorado','01-01-2010','12-31-2015',ozone_state_dict)
   
zip3_graph('800','01-01-2010','12-31-2015',ozone_zip3_dict)

In [ ]:
#print range_check(-105.002332,39.746389,1.4,'01-01-2010','12-31-2015',so2_state_dict)
#print range_check(-105.002332,39.746389,20,'01-01-2010','12-31-2015',so2_state_dict)
    
state_graph('Colorado','01-01-2010','12-31-2015',so2_state_dict)
    
zip3_graph('956','01-01-2010','12-31-2015',so2_zip3_dict)

In [ ]:
#print range_check(-105.002332,39.746389,1.4,'01-01-2010','12-31-2015',co_state_dict)
#print range_check(-105.002332,39.746389,20,'01-01-2010','12-31-2015',co_state_dict)
    
state_graph('Colorado','01-01-2010','12-31-2015',co_state_dict)
    
zip3_graph('200','01-01-2010','12-31-2015',co_zip3_dict)

In [ ]:
#print range_check(-105.002332,39.746389,1.4,'01-01-2010','12-31-2015',no2_state_dict)
#print range_check(-105.002332,39.746389,20,'01-01-2010','12-31-2015',no2_state_dict)
    
state_graph('Colorado','01-01-2010','12-31-2015',no2_state_dict)
    
zip3_graph('956','01-01-2010','12-31-2015',no2_zip3_dict)

In [ ]:
print range_check(-105.002332,39.746389,1.4,'10-01-2014','12-31-2014',no2_state_dict)